In [ ]:
%load_ext autoreload
%autoreload 2
%pylab inline

In [ ]:
%load_ext Cython
%load_ext line_profiler
%load_ext memory_profiler

In [ ]:
from tmqrfeed.manager import DataManager
from tmqralphas.strategy_v1_hedge_with_index import AlphaV1HedgeWithIndex
from tmqrstrategy.optimizers import OptimizerBase, OptimizerGenetic
            

In [ ]:
import logging
from tmqr.logs import log
logging.basicConfig(format='%(levelname)s: %(message)s', level=logging.INFO)

In [ ]:
dm = DataManager()

ALPHA_CONTEXT = {
    'name': 'AlphaV1Exposure_HedgedBy_V2_Index_CL_Long_Aug_21',
    'wfo_params': {
        'window_type': 'rolling',  # Rolling window for IIS values: rolling or expanding
        'period': 'M',  # Period of rolling window 'M' - monthly or 'W' - weekly
        'oos_periods': 2,  # Number of months is OOS period
        'iis_periods': 12,  # Number of months in IIS rolling window (only applicable for 'window_type' == 'rolling')
    },
    'wfo_optimizer_class': OptimizerBase,
    'wfo_optimizer_class_kwargs': {
        'nbest_count': 1,
        'nbest_fitness_method': 'max'
    },
    'wfo_opt_params': [
        ('direction', [1]),
    ],
    'wfo_members_count': 1,
    'wfo_costs_per_contract': 0.0,
    'wfo_scoring_type': 'netprofit',
    
    
    'context': {
        'index_hedge_name': 'US.CL_EXOSemiFuture_Delta25', # Name of the index used for hedge
        'index_hedge_direction': -1, # ALLOWED 1, -1, or even 0 - i.e. no hedge
        
        'instrument': 'US.CL',
        'v1_alphas': [
            #'CL_ContFut_Short_Strategy_DSP_LPBP_Combination__Bearish_Aug_21_custom',
            #'CL_ContFut_Long_Strategy_DSP_LPBP_Combination__Bullish_2_Aug_21_custom',
            'CL_ContFut_Long_Strategy_DSP_LPBP_Combination__Bullish_Aug_21_custom',
            #'CL_ContFut_Long_IchimokuCloud__Bullish_Jan17custom',
        ]
    }
}

alpha = AlphaV1HedgeWithIndex(dm, **ALPHA_CONTEXT)

alpha.run()

# Cumulative AlphaV1 + V2 Index equity

In [ ]:
figsize(15, 8)

equity = alpha.position.get_pnl_series()

alpha_equity = alpha.v1_equity.sum(axis=1)
hedge_equity = equity.equity_decision

hedge_equity.plot(label='V2 Hedge')
alpha_equity.plot(label='V1 alphas')

#Cumulative
(alpha_equity + hedge_equity).plot(label='Cumulative')

legend(loc=2)

# Framework v1 alpha exposures

In [ ]:

alpha.v1_exposure.sum(axis=1).plot()

# Framework v1 alphas  equities

In [ ]:
alpha.v1_equity.plot()


### Cumulative v1 alphas equity

In [ ]:
alpha.v1_equity.sum(axis=1).plot()

In [ ]:
alpha.save()